In [ ]:
"""
### okay so:
1. set up a way to continually play the game and save the networks
    play
    save
    train
    checkpoint
    
2. Think about network diversity

3. Think properly about the hyperparameters and search tree parameters

4. Think about how to stop other things interferring with the generation and training...

5. Read about bayesian q learning and maybe get back into bayesian a bit more in general
    it feels as though bayesian learning and search trees should go well together, measuring uncertainty as well
    as value

6. build a model loader and save game loader separately

7. after the latest stuff is all tested, move away from using chrome so it can be fully shut down!

8. think carefully about a nice structure for the code, in terms of where to sit each function...

"""


# okay so first thing is to improve the saving, build a model loader, build a trainer script, build a save game ifier

# Go through and clean all code and make it NICE to work with!


In [1]:
import random
import matplotlib.pyplot as plt
import datetime
import os
from pathlib import Path

import numpy as np
import copy
from importlib import reload
import torch
import ray
import pickle

import int_to_board
reload(int_to_board)

import game
reload(game)

import plotting
reload(plotting)

import mcts.mcts
reload(mcts.mcts)

import mcts.networks
reload(mcts.networks)

import mcts.agent
reload(mcts.agent)

import augmentor
reload(augmentor)

import raygent
reload(raygent)

from game import Patterns
from plotting import PatternPlotter

from mcts.mcts import Tree, Node
from mcts.networks import PatternsNet
from mcts.agent import Agent

from augmentor import StateAugmentor

from raygent import AgentWorker

rseed = 12387623
random.seed(rseed)
torch.manual_seed(rseed)
np.random.seed(rseed)

my_device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")


In [2]:
ray.init()


2025-07-21 17:20:47,490	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.5
Ray version:,2.42.1
Dashboard:,http://127.0.0.1:8265


In [3]:
### load the model:
network_path = os.path.join(Path.cwd(), 'saved_networks', 'v4', '2025_07_11_15_25.pt')
my_network = PatternsNet(in_channels=47, out_channels=128)
my_network.load_state_dict(torch.load(network_path, weights_only=True))

<All keys matched successfully>

In [4]:
### how long to keep up the training iterations:
NUM_IT = 1 # 1 validation, 15*30k games:

### Agent parameters:
NUM_TREES = 10
TARGET_GAMES = 10


### exploration schedule, to control number of searches with depth of tree:
SCHEDULE = [
    (0, 0), # explore randomly for the first moves:
    (1, 25), # limited search to avoid worst moves:
    (10, 300), # deeper search for deeper games:
]

### number of moves from terminal to save with:
SAVE_DEPTH = 5

### breadth-search parameters to favor narrower, deeper trees even early on:
TOPN = 6 # 6 best moves according to the puct scores:
RANDN = 4 # 4 additional random moves to favor exploration:

### how the next move is chosen. If Temp is not None, sampled from visit count distribution
SELECTION_TEMP = 1.0

agent_config = {
   # "agent_id": "agent",
    "network": my_network,
    "device": my_device,
    "num_trees": NUM_TREES,
    "target_games": TARGET_GAMES,
    "selection_temperature": SELECTION_TEMP,
    "topn": TOPN,
    "randn": RANDN,
    "save_depth": SAVE_DEPTH,
    "explore_steps_schedule": SCHEDULE,
    "debug": False,
}

### number of agents running as separate python processes at once:
NUM_RAYGENTS = 2


In [5]:
### start the ray processes:

futures = []

for _it in range(NUM_RAYGENTS):
    _config = agent_config.copy()
    _config["agent_id"] = f"raygent_{_it}"
    _rgent = AgentWorker.remote(_config)
    futures.append(_rgent.run.remote())

# Wait for completion and collect all games
all_completed_games = ray.get(futures)

# (Optional) Flatten or save
print("All agents completed!")

(AgentWorker pid=70524) Generating initial games:
(AgentWorker pid=70524) Evaluating tensor states...
(AgentWorker pid=70524) Provisioning inference to root nodes...
(AgentWorker pid=70524) 1 games have been completed!
(AgentWorker pid=70524) 2 games have been completed!
(AgentWorker pid=70524) 3 games have been completed!
(AgentWorker pid=50336) Generating initial games:
(AgentWorker pid=50336) Evaluating tensor states...
(AgentWorker pid=50336) Provisioning inference to root nodes...
(AgentWorker pid=50336) 1 games have been completed!
(AgentWorker pid=70524) 5 games have been completed! [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(AgentWorker pid=50336) 8 games have been completed! [repeated 12x across cluster]
All agents completed!
(AgentWorker pid=50336) 10 games have been completed!


In [7]:
total_tdata = 0

for _games in all_completed_games:
    total_tdata += len(_games[0])
    total_tdata += len(_games[-1])
    total_tdata += len(_games[1])

total_tdata

100

In [ ]:
print(f"{**agent_config}")

In [ ]:
len(all_completed_games[0][1])

In [ ]:
set([_x for _x in self.active_orthogonal_groups[color]
                                                  if self.active_board[_x] == color
                                                  if _x not in self.passive_orthogonal_groups[color]])

In [ ]:
def fun1(n):
    adder = {5, 6, 7, 8}
    start_set = {0, 1, 2, 3, 4}
    for _ in range(n):
        start_set.update(adder)
        start_set -= adder

def fun2(n):
    adder = {5, 6, 7, 8}
    start_set = {0, 1, 2, 3, 4}
    for _ in range(n):
        start_set |= adder
        start_set -= adder


In [ ]:
%prun fun2(100000)